In [1]:
import numpy as np
import csv    #加载csv包便于读取csv文件
import zipfile
import os
import json
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import datetime
import time
from datetime import timedelta

def date_2_timestamp(year,mon,day):
    tempString=str(year)+"-"+str(mon)+"-"+str(day)
    tempTime=time.strptime(tempString, "%Y-%m-%d")
    return time.mktime(tempTime)

def timestamp_2_date(un_time):
    return datetime.datetime.fromtimestamp(un_time)


def timestamp_removeDay_reduce(temp_timestamp):
    temp_date=timestamp_2_date(temp_timestamp)
    temp_timestamp=date_2_timestamp(temp_date.year,temp_date.month,1)
    return temp_timestamp

def timestamp_removeDay_add(temp_timestamp):
    temp_date=timestamp_2_date(temp_timestamp)
    if temp_date.month==12:
        return date_2_timestamp(temp_date.year+1,1,1)
    else:
        return date_2_timestamp(temp_date.year,temp_date.month+1,1)
    
def date_removeDay_reduce(temp_date):
    temp_timestamp=date_2_timestamp(temp_date.year,temp_date.month,1)
    return timestamp_2_date(temp_timestamp)

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

# 返回从最小到最大之间经过的时间戳，间隔为一个月
def max_min_2_timestampList(minDate,maxDate):    
    resList=[]
    resMap={}
    for single_date in daterange(minDate, maxDate):
        temp_date=date_removeDay_reduce(single_date)
        temp_timestamp=date_2_timestamp(temp_date.year,temp_date.month,temp_date.day)
        resMap[temp_timestamp]=1 

    for key in resMap.keys():
        resList.append(key)
    
    # print(resList)
    return sorted(resList)

def timestamp_2_string(temp_timestamp):
    temp_date=datetime.datetime.fromtimestamp(temp_timestamp)
    if temp_date.month<10:
        tempMonth="0"+str(temp_date.month)
    else:
        tempMonth=str(temp_date.month)
    return str(temp_date.year)+"-"+tempMonth

In [3]:
minStamp=1541142268
maxStamp=1674844823

minStamp_removeDay=timestamp_removeDay_reduce(minStamp)
maxStamp_removeDay=timestamp_removeDay_reduce(maxStamp)

minDate=datetime.datetime.fromtimestamp( minStamp_removeDay )
maxDate=datetime.datetime.fromtimestamp( maxStamp_removeDay )


# Exchange_defi

In [ ]:

def draw(mapPath,labelName):
    with open(mapPath, "rb") as tf:
        exchange_timeMap=pickle.load(tf)

    list_key=[]
    list_value=[]


    for tempTimestamp in max_min_2_timestampList(minDate,maxDate):
        if tempTimestamp not in exchange_timeMap:
            exchange_timeMap[tempTimestamp]=0
        
        list_key.append(timestamp_2_string(tempTimestamp))
        list_value.append( exchange_timeMap[tempTimestamp] )
        
    plt.plot(list_key, list_value,  markersize=20, label=labelName)
    
    # print(list_key)
    # print(list_value)



exchangeList=["UniswapV1","UniswapV2","UniswapV3", "BalancerV1","BalancerV2", "Curve", "SushiSwap", "ShibaSwap"]
for item in exchangeList:
    mapPathDir="/mnt/sde1/peilin_defi/data/MEVTransactions/dict/flashbotsCount/"
    mapPath=mapPathDir+item+".map"
    draw(mapPath,item)

list_key_length=len(max_min_2_timestampList(minDate,maxDate))

plt.legend(fontsize=16)
plt.xticks(range(1,list_key_length,8),rotation=45,fontsize=20)
plt.yticks(fontsize=20)
plt.yscale('log')
plt.ylabel('Monthly MEV TXs', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.tight_layout()
plt.grid(True, color='#666666', linestyle = ":", linewidth = "1")

plt.savefig("/mnt/sde1/peilin_defi/data/MEVTransactions/photo/deFi_exchange.pdf",bbox_inches='tight')

# Lending_defi

In [ ]:
def draw(mapPath,labelName):
    with open(mapPath, "rb") as tf:
        exchange_timeMap=pickle.load(tf)

    list_key=[]
    list_value=[]


    for tempTimestamp in max_min_2_timestampList(minDate,maxDate):
        if tempTimestamp not in exchange_timeMap:
            exchange_timeMap[tempTimestamp]=0
        
        list_key.append(timestamp_2_string(tempTimestamp))
        list_value.append( exchange_timeMap[tempTimestamp] )
        
    plt.plot(list_key, list_value,  markersize=20, label=labelName)
    


lendingList=["AaveV1","AaveV2", "MakerDAO", 'Compound', "Cream"]

for item in lendingList:
    mapPathDir="/mnt/sde1/peilin_defi/data/MEVTransactions/dict/flashbotsCount/"
    mapPath=mapPathDir+item+".map"
    draw(mapPath,item)

list_key_length=len(max_min_2_timestampList(minDate,maxDate))

plt.legend(fontsize=16)
plt.xticks(range(1,list_key_length,8),rotation=45,fontsize=20)
plt.yticks(fontsize=20)
plt.yscale('log')
plt.ylabel('Monthly MEV TXs', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.tight_layout()
plt.grid(True, color='#666666', linestyle = ":", linewidth = "1")

plt.savefig("/mnt/sde1/peilin_defi/data/MEVTransactions/photo/deFi_lending.pdf",bbox_inches='tight')

# Exchange_nft

In [ ]:
def draw(mapPath,labelName):
    with open(mapPath, "rb") as tf:
        exchange_timeMap=pickle.load(tf)

    list_key=[]
    list_value=[]


    for tempTimestamp in max_min_2_timestampList(minDate,maxDate):
        if tempTimestamp not in exchange_timeMap:
            exchange_timeMap[tempTimestamp]=0
        
        list_key.append(timestamp_2_string(tempTimestamp))
        list_value.append( exchange_timeMap[tempTimestamp] )
    
    # 消除opensea的下降线
    if labelName=="OpenSea":
        tempMaxList=[]
        for i in range(1,len(list_value)-1):
            front=list_value[i-1]
            cur=list_value[i]
            next=list_value[i+1]
            if cur>front and cur>next:
                tempMaxList.append(i)
                
        print(tempMaxList)
                
        
        temp_list_key=[]
        temp_list_value=[]
        
        for i in range(len(list_key)):
            if i<tempMaxList[-1]:
                temp_list_key.append(list_key[i])
                temp_list_value.append(list_value[i])
                
        list_key=temp_list_key
        list_value=temp_list_value
        
            
        
    plt.plot(list_key, list_value,  markersize=20, label=labelName)
    


# lendingList=["AaveV1","AaveV2", "MakerDAO", 'Compound', "Cream"]
exchange_nft_list=["OpenSea", "LooksRare", "X2Y2", "Blur"]


for item in exchange_nft_list:
    mapPathDir="/mnt/sde1/peilin_defi/data/MEVTransactions/dict/flashbotsCount/"
    mapPath=mapPathDir+item+".map"
    draw(mapPath,item)

list_key_length=len(max_min_2_timestampList(minDate,maxDate))

plt.legend(fontsize=16)
plt.xticks(range(1,list_key_length,8),rotation=45,fontsize=20)
plt.yticks(fontsize=20)
plt.yscale('log')
plt.ylabel('Monthly MEV TXs', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.tight_layout()
plt.grid(True, color='#666666', linestyle = ":", linewidth = "1")

plt.savefig("/mnt/sde1/peilin_defi/data/MEVTransactions/photo/nft_exchange.pdf",bbox_inches='tight')

# Lending_nft

In [ ]:
def draw(mapPath,labelName):
    with open(mapPath, "rb") as tf:
        exchange_timeMap=pickle.load(tf)

    list_key=[]
    list_value=[]


    for tempTimestamp in max_min_2_timestampList(minDate,maxDate):
        if tempTimestamp not in exchange_timeMap:
            exchange_timeMap[tempTimestamp]=0
        
        list_key.append(timestamp_2_string(tempTimestamp))
        list_value.append( exchange_timeMap[tempTimestamp] )
        
    plt.plot(list_key, list_value,  markersize=20, label=labelName)
    


# lendingList=["AaveV1","AaveV2", "MakerDAO", 'Compound', "Cream"]
# exchange_nft_list=["OpenSea", "LooksRare", "X2Y2", "Blur"]
lending_nft_list=["NFTfiV1","NFTfiV2", "Arcade", "BendDAO"]

for item in lending_nft_list:
    mapPathDir="/mnt/sde1/peilin_defi/data/MEVTransactions/dict/flashbotsCount/"
    mapPath=mapPathDir+item+".map"
    draw(mapPath,item)

list_key_length=len(max_min_2_timestampList(minDate,maxDate))

plt.legend(fontsize=16)
plt.xticks(range(1,list_key_length,8),rotation=45,fontsize=20)
plt.yticks(fontsize=20)
plt.yscale('log')
plt.ylabel('Monthly MEV TXs', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.tight_layout()
plt.grid(True, color='#666666', linestyle = ":", linewidth = "1")

plt.savefig("/mnt/sde1/peilin_defi/data/MEVTransactions/photo/nft_lending.pdf",bbox_inches='tight')

# 饼状图：Flashbots的总交易(或Bundle)中，20个协议的比例，同时涉及多个的另外算一个叫Multiple协议的，所以总共有22个（20+1+1个啥都不属于的）

In [8]:
# flashbotsTxMap_length=11915557
flashbotsTxMap_length=23300343

# pathDir="/mnt/sde1/peilin_defi/data/MEVTransactions/dict/flashbotsCount_1549_1649/"
pathDir="/mnt/sde1/peilin_defi/data/MEVTransactions/dict/flashbotsCount/"

marketMap={}
countSum=0
for fileName in os.listdir(pathDir):
    filePath=pathDir+fileName
    marketType=fileName.split(".")[0]
    
    tempTimeMap={}
    with open(filePath, "rb") as tf:
        tempTimeMap=pickle.load(tf)
        
    count=sum(tempTimeMap.values())
    countSum+=count
    marketMap[marketType]=count/flashbotsTxMap_length
    
    
marketMap["Others"]=(flashbotsTxMap_length-countSum)/flashbotsTxMap_length

In [10]:
# 排序
reverseMap={}
for key,value in marketMap.items():
    reverseMap[value]=key
    
valueList=list(marketMap.values())
valueList.sort(reverse=False)
print(valueList)

keyList=[]
for item in valueList:
    keyList.append(reverseMap[item])

print(keyList)


[1.072945578526462e-06, 3.1759189124383276e-06, 4.291782314105848e-06, 1.7467554018410803e-05, 3.89693834120811e-05, 5.154430559241124e-05, 8.922615431026058e-05, 0.00015167158698050067, 0.0001557916980020423, 0.000332441458050639, 0.0004164745557608315, 0.001426760112501348, 0.0015920795672407055, 0.0019789408250342065, 0.002173530235155766, 0.003072100698260107, 0.0030853193877875534, 0.005248163084981195, 0.007403839505710281, 0.06767535568038634, 0.12693989097070374, 0.15414567073111327, 0.6239962218581933]
['NFTfiV1', 'Cream', 'Arcade', 'AaveV1', 'BendDAO', 'Multiple', 'NFTfiV2', 'MakerDAO', 'UniswapV1', 'LooksRare', 'Compound', 'X2Y2', 'AaveV2', 'BalancerV1', 'Blur', 'OpenSea', 'BalancerV2', 'Curve', 'ShibaSwap', 'SushiSwap', 'UniswapV3', 'UniswapV2', 'Others']


In [ ]:
num1=1
num2=1
num3=2
num4=0

keys = list(keyList)
values = np.array(valueList)
# patches, texts = plt.pie(values,colors=rgb_list)

def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = pct*total/100.0
        if pct<6.767535209655762:
            return ''
        else:
            return '{p:.2f}%'.format(p=pct)
    
    
    return my_autopct
patches, texts,_ = plt.pie(values,autopct=make_autopct(values))

# labels = ['{} {:1.2f}% ({})'.format(i, j, v) for i, j, v in
#             zip(keys, 100. * values / values.sum(), values)]  # 构造图例数据
labels = ['{} ({:1.4f}%) '.format(i, j) for i, j in
            zip(keys, 100. * values / values.sum())]  # 构造图例数据

patches, labels, dummy = zip(*sorted(
    zip(patches, labels, values), key=lambda x: x[2], reverse=True))

plt.legend(patches, labels, fontsize=10,bbox_to_anchor=(num1, num2), loc=num3, borderaxespad=num4,ncol=2)
plt.tight_layout()
plt.savefig("/mnt/sde1/peilin_defi/data/MEVTransactions/photo/MEVTotal.pdf",bbox_inches='tight')